# Robex Cashflow Analysis

In [1]:
# Hot to increase margin 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
# pip install jupyter_dash

In [3]:
# Importing libraries required
import pandas as pd
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as pyo
import plotly.graph_objs as go

pd.set_option('display.float_format', lambda x: '%.3f' % x)

Dates = pd.date_range(start = ('2020-01-01'), end = '2030-01-01', freq = 'Y')

## 1. Scenario Building

In [4]:
### Scenario 1: EBITDA of 60M for 7 years at 3% inflation
### Scenario 2: EBITDA of 60M for 10 years at 3% inflation

### Scenario 3: EBITDA of 80M for 7 years at 3% inflation
### Scenario 4: EBITDA of 80M for 10 years at 3% inflation

### Scenario 5: EBITDA of 100M for 10 years at 3% inflation + 30M loan at 6.5% with a term of 10 years
### Scenario 6: EBITDA of 125M for 10 years at 3% inflation + 30M loan at 6.5% with a term of 10 years

### Scenario 7: Initial CAPITAL of 80M --> 30M payable in cash and 50M loan at 6.5% with a term of 10 years      
###             EBITDA of 125M

In [5]:
# Basic scenario creation
S1 = np.repeat(60, 7)
S2 = np.repeat(60, 10)
S3 = np.repeat(80, 7)
S4 = np.repeat(80, 10)

S5 = np.repeat(100, 10)
S5[0] = 100+30
S6 = np.repeat(125, 10)
S6[0] = 125+30

S7 = np.repeat(125, 10)

In [6]:
# Scenario 7 cash outflow
capital_payable = 30
S7[0] = 125 - capital_payable + 50

# Loan payments of 50M:
interest_rate=0.065/12
mortgage_amount=50
n_periods = 10*12

m_payment = npf.pmt(interest_rate, n_periods, mortgage_amount)
payment1 = round(m_payment * 12,2)
payment1 = np.repeat(payment1,10)




[-6.81 -6.81 -6.81 -6.81 -6.81 -6.81 -6.81 -6.81 -6.81 -6.81] [145 125 125 125 125 125 125 125 125 125]


In [7]:
# Creating a simple inflation timeline
inflation7 = np.repeat(1.03, 7)
inflation7 = (lambda inf: inf**range(1,8))(inflation7)


inflation10 = np.repeat(1.03, 10)
inflation10 = (lambda inf: inf**range(1,11))(inflation10)


array([1.03      , 1.0609    , 1.092727  , 1.12550881, 1.15927407,
       1.1940523 , 1.22987387, 1.26677008, 1.30477318, 1.34391638])

In [8]:
# Loan Payments

interest_rate=0.065/12
mortgage_amount=30
n_periods = 10*12

m_payment = npf.pmt(interest_rate, n_periods, mortgage_amount)

payment = round(m_payment * 12,2)
payment = np.repeat(payment,10)


array([-4.09, -4.09, -4.09, -4.09, -4.09, -4.09, -4.09, -4.09, -4.09,
       -4.09])

## 2. Multiplying the EBITDA with the inlfation

In [9]:
S1 = pd.DataFrame(np.append(S1, [0,0,0]) * inflation10)
S2 = pd.DataFrame(S2 * inflation10)
S3 = pd.DataFrame(np.append(S3, [0,0,0]) * inflation10)
S4 = pd.DataFrame(S4 * inflation10)

In [10]:
S5 = pd.DataFrame((S5 * inflation10) + payment)

S6 = pd.DataFrame((S6 * inflation10) + payment)

S7 = pd.DataFrame((S7 * inflation10) + payment1)


,0
0,142.540
1,125.802
2,129.781
3,133.879
4,138.099
5,142.447
6,146.924
7,151.536
8,156.287
9,161.180


In [11]:
df = pd.concat([S1, S2, S3, S4, S5, S6, S7], axis=1)

In [12]:
df.columns = ['Scenario 1', 'Scenario 2', 'Scenario 3', 'Scenario 4', 'Scenario 5', 'Scenario 6', 'Scenario 7']
df.index = Dates



,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7
2020-12-31,61.800,61.800,82.400,82.400,129.810,155.560,142.540
2021-12-31,63.654,63.654,84.872,84.872,102.000,128.522,125.802
2022-12-31,65.564,65.564,87.418,87.418,105.183,132.501,129.781
2023-12-31,67.531,67.531,90.041,90.041,108.461,136.599,133.879
2024-12-31,69.556,69.556,92.742,92.742,111.837,140.819,138.099
2025-12-31,71.643,71.643,95.524,95.524,115.315,145.167,142.447
2026-12-31,73.792,73.792,98.390,98.390,118.897,149.644,146.924
2027-12-31,0.000,76.006,0.000,101.342,122.587,154.256,151.536
2028-12-31,0.000,78.286,0.000,104.382,126.387,159.007,156.287
2029-12-31,0.000,80.635,0.000,107.513,130.302,163.900,161.180


In [13]:
# Example 1: Simple Bar charts
# Example 2: more complex with traces that seperates medals between gold, silver and bronze


trace1 = go.Bar(
    y=df.index,  # NOC stands for National Olympic Committee
    x=df['Scenario 1'],
    name = 'Scenario 1',
    orientation='h',
    marker=dict(color='#17175c') # set the marker color to gold
)

# Silver Medals
trace2 = go.Bar(
    y=df.index,
    x=df['Scenario 2'],
    name='Scenario 2',
    orientation='h',
    marker=dict(color='#005697') # set the marker color to silver
)

# Bronze Medals
trace3 = go.Bar(
    y=df.index,
    x=df['Scenario 3'],
    name='Scenario 3',
    orientation='h',
    marker=dict(color='#008fa7') # set the marker color to bronze
)

trace4 = go.Bar(
    y=df.index,
    x=df['Scenario 4'],
    name='Scenario 4',
    orientation='h',
    marker=dict(color='#2fc498') # set the marker color to bronze
)

trace5 = go.Bar(
    y=df.index,
    x=df['Scenario 5'],
    name='Scenario 5',
    orientation='h',
    marker=dict(color='red') # set the marker color to bronze
)


trace6 = go.Bar(
    y=df.index,
    x=df['Scenario 6'],
    name='Scenario 6',
    orientation='h',
    marker=dict(color='purple') # set the marker color to bronze
)

trace7 = go.Bar(
    y=df.index,
    x=df['Scenario 7'],
    name='Scenario 7',
    orientation='h',
    marker=dict(color='orange') # set the marker color to bronze
)

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7]

layout = go.Layout(
    title='Robex Scenario 1-7 CashFlow Distribution',
    barmode='relative' # This will stack our bars instead of having them seperate !! 
)


fig = go.Figure(data=data, layout=layout)
pyo.plot(fig)

'temp-plot.html'

In [14]:
def robex_npv(rate):
    
    robex_npv = []
    disc = []
    
    for i in range(7):
        npv = round(npf.npv(rate, df.iloc[:, i]), 2)
        
        disc.append(rate)
        robex_npv.append(npv)
        
    return(pd.DataFrame([robex_npv, disc]).T)


In [15]:
index = ['Scenario 1', 'Scenario 2', 'Scenario 3', 'Scenario 4', 'Scenario 5', 'Scenario 6', 'Scenario 7']
col_name = ['NPV', 'Discount Rate']

npv_robex1 = robex_npv(0.065)
npv_robex1.columns = col_name
npv_robex1.index = index

npv_robex2 = robex_npv(0.07)
npv_robex2.columns = col_name
npv_robex2.index = index

npv_robex3 = robex_npv(0.075)
npv_robex3.columns = col_name
npv_robex3.index = index

npv_robex4 = robex_npv(0.08)
npv_robex4.columns = col_name
npv_robex4.index = index

robex_final_npv = npv_robex1.append([npv_robex2, npv_robex3, npv_robex4])


,NPV,Discount Rate
Scenario 1,392.210,0.065
Scenario 2,534.170,0.065
Scenario 3,522.950,0.065
Scenario 4,712.230,0.065
Scenario 5,889.870,0.065
Scenario 6,1112.450,0.065
Scenario 7,1081.320,0.065
Scenario 1,387.000,0.070
Scenario 2,523.750,0.070
Scenario 3,516.000,0.070


In [16]:
robex_final_npv.reset_index(inplace = True)

In [17]:
df2 = robex_final_npv


,index,NPV,Discount Rate
0,Scenario 1,392.210,0.065
1,Scenario 2,534.170,0.065
2,Scenario 3,522.950,0.065
3,Scenario 4,712.230,0.065
4,Scenario 5,889.870,0.065
5,Scenario 6,1112.450,0.065
6,Scenario 7,1081.320,0.065
7,Scenario 1,387.000,0.070
8,Scenario 2,523.750,0.070
9,Scenario 3,516.000,0.070


In [ ]:

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
from jupyter_dash import JupyterDash
import dash_auth # used to create username and password for dashboard publication

# df = pd.read_csv(shortcut + 'data/gapminderDataFiveYear.csv')
# colors1 = ['#222831', '#393e46', '#00adb5', '#14274e', '#394867', '#9ba4b4']

# Robex Logo url link
imgg = 'https://www.globenewswire.com/news-release/logo/531621/0/531621.png?lastModified=10%2F09%2F2020%2013%3A07%3A21&size=2&v=2040525'

# Username and password:
username_password = [['username', 'password'], ['Dubeau', 'Robex']]

# Begining of Dashboard creation
# app = dash.Dash()
app = JupyterDash()

# auth = dash_auth.BasicAuth(app, username_password)
# server = app.server

# https://dash.plot.ly/dash-core-components/dropdown
# We need to construct a dictionary of dropdown values for the years
discount = []
for rate in df2['Discount Rate'].unique():
    discount.append({'label':str(rate),'value':rate})

app.layout = html.Div([
    #html.H1('Robex', style={'background-image': imgg}),
    html.Img(src = imgg),
    dcc.Graph(id='graph'),
    dcc.Dropdown(id='Rates', options=discount, value=df2['Discount Rate'][1], style = {'color':'#008fa7'})
],
style={'width':1600, 'height':600, 'color':'#FFCD49', 'border':'2px #030C14 solid', 'background': '#030C14'})


@app.callback(Output('graph', 'figure'),
              [Input('Rates', 'value')])

def update_figure(selected_rate):
    filtered_df = df2[df2['Discount Rate'] == selected_rate]
    traces = []
    
    for continent_name in filtered_df['index'].unique():
        df_by_continent = filtered_df[filtered_df['index'] == continent_name]
        traces.append(go.Bar(
            x=df_by_continent['NPV'],
            y=df_by_continent['index'],
            text=df_by_continent['index'],
            orientation = 'h',
            name=continent_name,
            marker=dict(color='#008fa7')
        ))

    return {
        'data': traces,
        'layout': go.Layout(
            xaxis={'title': 'Net Present Value', 'color': '#FFCD49'},
            yaxis= {'color': '#FFCD49'},
            title = 'Robex Cash Flow Analysis',
            titlefont = {'color': '#FFCD49'},
            plot_bgcolor= 'rgba(3,12,20, 0.8)',
            paper_bgcolor= 'rgba(3,12,20, 0.8)',
            legend = {'bordercolor': '#FFCD49', 'borderwidth': 2, 'font':{'color': '#FFCD49'}},
            hovermode='closest'
        )
    }


app.run_server(mode='inline')



# trace1 = go.Bar(
#     y=df.index,  # NOC stands for National Olympic Committee
#     x=df['Scenario 1'],
#     name = 'Scenario 1',
#     orientation='h',
#     marker=dict(color='#17175c') # set the marker color to gold
# )


# if __name__ == '__main__':
#     app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
